# Introduction

In this notebook, we shall be using Siamese Network in order to build a model to perform the task of face verification for a given character.

The paper referred to for performing this experiment is [linked here](https://proceedings.neurips.cc/paper/1993/file/288cc0ff022877bd3df94bc9360b9c5d-Paper.pdf).

In [1]:
from siameseDataset import *
from loss_func import *
from siameseModel import *
import torch
from torch import nn as nn
import pandas as pd
torch.autograd.set_detect_anomaly(True)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_path = "/home/vinayak/cleaned_anime_faces"
model_save_path = "/home/vinayak/anime_face_recognition/enet_model.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
partition = {}
split_info = pd.read_csv(f"/home/vinayak/anime_face_recognition/data.csv")
partition["train"] = list(split_info[split_info.label == "train"].images)
random.shuffle(partition["train"])
partition["validation"] = list(split_info[split_info.label == "valid"].images)

In [4]:
# https://omoindrot.github.io/triplet-loss#strategies-in-online-mining

In [5]:
# Create a training dataset and use it to create a training_generator
training_set = siameseDataset(partition['train'])
training_generator = torch.utils.data.DataLoader(training_set, batch_size = 1)
# training_set.show_sample()

In [6]:
# Create a training dataset and use it to create a validation_generator
validation_set = siameseDataset(partition['validation'], dtype = "validation")
validation_generator = torch.utils.data.DataLoader(validation_set, batch_size = 1)
# validation_set.show_sample()

In [7]:
# Create the model and move it to appropriate device (i.e. cuda if gpu is available)
model = enet_model().to(DEVICE)

In [8]:
# Define the loss function to be used for training
loss_func = batchHardTripletLoss().to(DEVICE)

In [9]:
# Define a learning rate and create an optimizer for training the model 
# (Adam with default momentum should be good)
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define a learning rate scheduler so that you reduce the learning rate
# As you progress across multiple epochs
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 3, factor = 0.2, threshold = 1e-5)

In [10]:
train_losses = []
valid_losses = []

n_epochs = 5
n_train_batches = len(training_generator)
n_valid_batches = len(validation_generator)
PRINT_PROGRESS = 10

round_off = lambda x: round(x, 5)

# Loop over number of epochs
for epch in range(n_epochs):
    
    # Initialize the loss values to zero at the beginning of the epoch
    train_loss = 0.
    valid_loss = 0.

    # Train for an epoch
    for idx, (images, labels) in enumerate(training_generator, start = 1):
        images, labels = images[0].to(DEVICE), labels.to(DEVICE)
        feature_vectors = model(images)
        loss = loss_func(feature_vectors, labels)
        loss.backward()
        optimizer.step()
        
        batch_loss = round_off(loss.item())
        train_loss += batch_loss
        
        if (idx % PRINT_PROGRESS == 0) or (idx == 0) or (idx == n_train_batches - 1):
            print(f"Epoch: {(epch + 1):<4}| Batch Number: {idx:<4}| Current Batch Loss: {batch_loss:<7}| Average Train Loss: {round_off(train_loss / idx):<7}")
    
    # Validate after the trained epoch
    for images, labels in validation_generator:
        images, labels = images[0].to(DEVICE), labels.to(DEVICE)
        with torch.no_grad():
            feature_vectors = model(images)
            loss = loss_func(feature_vectors, labels)
            valid_loss += round_off(loss.item())
    
    # Average the train and valid losses across all batches and save it to our array
    train_loss = round_off(train_loss / n_train_batches)
    valid_loss = round_off(valid_loss / n_valid_batches)
    
    print(f"####################### End of Epoch {epch + 1} #######################")
    print(f"Epoch: {(epch + 1):<4}| Train Loss: {train_loss:<7}| Valid Loss: {valid_loss:<7}")
    print()
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    # Check the valid loss and reduce learning rate as per the need
    scheduler.step(valid_loss)

Epoch: 1   | Batch Number: 10  | Current Batch Loss: 2.7512 | Average Train Loss: 2.32116
Epoch: 1   | Batch Number: 20  | Current Batch Loss: 1.92464| Average Train Loss: 2.1912 
Epoch: 1   | Batch Number: 30  | Current Batch Loss: 1.64174| Average Train Loss: 2.12889
Epoch: 1   | Batch Number: 40  | Current Batch Loss: 1.53397| Average Train Loss: 2.05252
Epoch: 1   | Batch Number: 50  | Current Batch Loss: 1.22877| Average Train Loss: 1.96055
Epoch: 1   | Batch Number: 54  | Current Batch Loss: 1.37387| Average Train Loss: 1.92623
####################### End of Epoch 1 #######################
Epoch: 1   | Train Loss: 1.91822| Valid Loss: 1.42327

Epoch: 2   | Batch Number: 10  | Current Batch Loss: 1.40383| Average Train Loss: 1.44146
Epoch: 2   | Batch Number: 20  | Current Batch Loss: 1.34176| Average Train Loss: 1.35352
Epoch: 2   | Batch Number: 30  | Current Batch Loss: 1.14791| Average Train Loss: 1.31153
Epoch: 2   | Batch Number: 40  | Current Batch Loss: 1.12557| Average Tr

In [11]:
# Save the losses to a loss_history.csv file on the disk
history = pd.DataFrame({"train_loss": train_losses, "valid_loss":valid_losses})
history.to_csv("loss_history.csv", index = False)

In [12]:
# Save the trained model to our disk
torch.save(model.state_dict(), model_save_path)